# VESC Motor Controller - Basic Usage

This notebook demonstrates basic usage of the VESC student API for reading telemetry data and controlling the motor.

## Setup

First, let's import the student API and create a connection to the VESC controller.

In [ ]:
# Setup Python path to find student_api from parent directory
import sys
import os
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))

from student_api import VESCStudentAPI
import time

vesc_api = VESCStudentAPI() # Create and start VESC API

if vesc_api.start():
    time.sleep(6)  # Wait for controller discovery
    
    connected_controllers = vesc_api.get_connected_controllers() # Check if VESC controller 74 is connected
    vesc = vesc_api.get_controller(74)
    
    if 74 in connected_controllers and vesc.is_connected():
        voltage = vesc.get_input_voltage()
        print(f"✅ Connected to VESC controller! Battery: {voltage:.1f}V")
    else:
        print("❌ VESC controller not found or not responding! Check that it is powered ON.")
else:
    print("❌ Failed to start VESC system")

## Reading Basic Telemetry

Let's read some basic motor telemetry data:

##### NOTE: If the motor is not spinning, the RPM, Current, and Duty Cycle will all read 0

In [2]:
# Read current motor status
rpm = vesc.get_rpm()
current = vesc.get_motor_current()
duty_cycle = vesc.get_duty_cycle()
voltage = vesc.get_input_voltage()

print(f"Input Voltage: {voltage} V")
print(f"Motor RPM: {rpm}")
print(f"Motor Current: {current} A")
print(f"Duty Cycle: {duty_cycle}")

Input Voltage: 25.2 V
Motor RPM: 0
Motor Current: 0.0 A
Duty Cycle: 0.0


## Reading Temperature Data

Monitor motor and controller temperatures:

##### NOTE: The Lectec vehicles do not have temperature sensors in the motor

In [ ]:
# Read temperature data
fet_temp = vesc.get_fet_temperature()
motor_temp = vesc.get_motor_temperature()

print(f"Controller (FET) Temperature: {fet_temp}°C")
print(f"Motor Temperature: {motor_temp}°C")

# Check if temperatures are safe
if fet_temp and fet_temp > 80:
    print("⚠️ WARNING: Controller temperature is high!")
if motor_temp and motor_temp > 100:
    print("⚠️ WARNING: Motor temperature is high!")

## Power and Energy Monitoring

Monitor power consumption and energy usage:

##### NOTE: If the motor is not moving, Input current will be 0. If the vehcile has not been used since startup, energy will also = 0.

In [ ]:
# Read power and energy data
input_current = vesc.get_input_current()
amp_hours = vesc.get_amp_hours_consumed()
watt_hours = vesc.get_watt_hours_consumed()

print(f"Input Current: {input_current} A")
print(f"Energy Consumed: {amp_hours} Ah, {watt_hours} Wh")

# Calculate approximate power
if voltage and input_current:
    power = voltage * input_current
    print(f"Approximate Power: {power:.2f} W")

## Live Data Monitoring

Get all telemetry data at once and display it continuously:

In [ ]:
# Monitor live data for 10 seconds
import time
from IPython.display import clear_output

print("Monitoring live data for 10 seconds...")
start_time = time.time()

while time.time() - start_time < 10:
    clear_output(wait=True)
    
    # Get all telemetry at once
    data = vesc.get_all_telemetry()
    
    print("=== VESC Live Data ===")
    print(f"RPM: {data['motor']['rpm']}")
    print(f"Current: {data['motor']['current']} A")
    print(f"Duty Cycle: {data['motor']['duty_cycle']}")
    print(f"Voltage: {data['power']['input_voltage']} V")
    print(f"FET Temp: {data['temperatures']['fet']}°C")
    print(f"Motor Temp: {data['temperatures']['motor']}°C")
    
    time.sleep(0.5)

print("Live monitoring complete!")

## Safe Motor Control

**⚠️ WARNING: Only run these commands when it's safe to do so!**

The following cells demonstrate motor control. Make sure the motor is properly secured and safe to operate.

In [4]:
# RUN THIS CELL FIRST BEFORE CONTINUING
# SAFETY CHECK - Uncomment the next line to enable motor control
# MOTOR_CONTROL_ENABLED = True
MOTOR_CONTROL_ENABLED = True

if not MOTOR_CONTROL_ENABLED:
    print("Motor control is DISABLED for safety.")
    print("Set MOTOR_CONTROL_ENABLED = True to enable motor commands.")

In [18]:
if MOTOR_CONTROL_ENABLED:
    # Set a small duty cycle (10%)
    print("Setting duty cycle to 10%...")
    success = vesc.set_duty_cycle(0.2)
    print(f"Command sent: {success}")
    
    # Wait and monitor
    time.sleep(1)
    rpm = vesc.get_rpm()
    print(f"Current RPM: {rpm}")
    
    # Stop the motor
    print("Stopping motor...")
    vesc.set_duty_cycle(0)
    print("Motor stopped.")
else:
    print("Motor control commands skipped (safety mode). Set MOTOR_CONTROL_ENABLED = True if desired.")

Setting duty cycle to 10%...
Command sent: True
Current RPM: 4241
Stopping motor...
Motor stopped.


## Current Control

**⚠️ WARNING: Only run these commands when it's safe to do so!**

Current control allows precise motor torque control. The VESC will maintain the specified current regardless of load or speed.

In [19]:
if MOTOR_CONTROL_ENABLED:
    # Set a small motor current (2 A)
    print("Setting motor current to 2.0 A...")
    success = vesc.set_current(2.0)
    print(f"Command sent: {success}")
    
    # Wait and monitor current values
    time.sleep(1)
    motor_current = vesc.get_motor_current()
    input_current = vesc.get_input_current()
    voltage = vesc.get_input_voltage()
    
    print(f"Motor Current: {motor_current} A")
    print(f"Input Current: {input_current} A")
    print(f"Input Voltage: {voltage} V")
    
    # Stop the motor
    print("Stopping motor...")
    vesc.set_current(0)
    print("Motor stopped.")
else:
    print("Motor control commands skipped (safety mode). Set MOTOR_CONTROL_ENABLED = True if desired.")

Setting motor current to 2.0 A...
Command sent: True
Motor Current: 0.4 A
Input Current: 0.3 A
Input Voltage: 25.2 V
Stopping motor...
Motor stopped.


## Brake Current Control

**⚠️ WARNING: Only run these commands when it's safe to do so!**

Brake current control applies regenerative braking to the motor. The motor acts as a generator, feeding energy back to the battery.

In [27]:
if MOTOR_CONTROL_ENABLED:

    # Apply brake current (5 A) for 5 Seconds
    #A loop to keep track of time from 1-5
    for i in range(5):
        success = vesc.set_brake_current(5.0) #set the brake current to -5
        print("Motor Current: ", vesc.get_motor_current()) #Print the Motor Current
        print("RPM: ", vesc.get_rpm()) #Print the RPM
        time.sleep(1) #Pause for 1 second
        ++i #Increment the counter of the loop, 1s has passed
    
    # Stop braking
    print("Releasing brake...")
    vesc.set_brake_current(0)
    

else:
    print("Motor control commands skipped (safety mode). Set MOTOR_CONTROL_ENABLED = True if desired.")

Motor Current:  0.0
RPM:  0
Motor Current:  0.0
RPM:  5
Motor Current:  1.4
RPM:  167
Motor Current:  2.9
RPM:  162
Motor Current:  0.0
RPM:  -12
Releasing brake...


## Cleanup

Always ensure the motor is stopped when finished:

In [ ]:
# Ensure motor is stopped
vesc.set_duty_cycle(0)
print("Session complete. Motor stopped.")